# 1. 데이터 로드

In [1]:
import os
import io
import json
import boto3
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone
import mlflow
import mlflow.keras
from joblib import load
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

from io import BytesIO
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = "raw-data-bucket-moasic-mlops-4"

s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

now_utc = datetime.utcnow().replace(minute=0, second=0, microsecond=0)
start_utc = now_utc - timedelta(hours=30)
date_list = pd.date_range(start=start_utc.date(), end=now_utc.date()).strftime("%Y-%m-%d").tolist()

dfs = []

for date in date_list:
    prefix = f"upbit_ticker/{date}/"
    response = s3_client.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix)

    if "Contents" not in response:
        continue

    for obj in response["Contents"]:
        key = obj["Key"]
        if key.endswith("/"):
            continue
        
        file_obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
        df_temp = pd.read_parquet(BytesIO(file_obj['Body'].read()))
        df_temp['event_timestamp'] = pd.to_datetime(df_temp['timestamp'], unit='ms') + pd.Timedelta(hours=9)

        start_kst = start_utc + timedelta(hours=9)
        end_kst = now_utc + timedelta(hours=9)
        df_temp = df_temp[(df_temp['event_timestamp'] >= start_kst) & (df_temp['event_timestamp'] <= end_kst)]
        
        if not df_temp.empty:
            dfs.append(df_temp)
            print(f"Loaded (30h range): {key}")

if len(dfs) == 0:
    print("30시간 범위 내 parquet 파일이 없습니다.")
    df = pd.DataFrame()
else:
    df = pd.concat(dfs, ignore_index=True) 
    print("All Parquet files concatenated. Shape:", df.shape)

if not df.empty:
    # 시 단위로 내림 (정시 기준)
    df['event_hour'] = df['event_timestamp'].dt.floor('H')

    coins = ['KRW-BTC', 'KRW-DOGE', 'KRW-ETH', 'KRW-XRP']
    coin_dfs = {}

    for coin in coins:
        # 각 정시별 첫 거래 선택
        coin_df = df[df['market'] == coin].groupby('event_hour').first().reset_index()
        coin_dfs[coin] = coin_df
        print(f"{coin} shape:", coin_df.shape)

Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T11.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T12.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T13.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T14.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T15.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T16.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T17.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T18.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T19.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T20.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T21.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T22.parquet
Loaded (30h range): upbit_ticker/2025-09-25/upbit_ticker_20250925T23.parquet

/var/folders/_d/d0lslv953psc0xrjcq10dlk80000gn/T/ipykernel_4175/484494982.py:71: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['event_hour'] = df['event_timestamp'].dt.floor('H')


# 2. 데이터 전처리

In [2]:
import os
import io
import joblib
# import mlflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# artifacts 폴더 경로
ARTIFACT_DIR = "artifacts"

# ============================= 
# 3. KRW-BTC 필터링 + 리샘플링
# =============================

# 비트코인 선택
df_btc = coin_dfs['KRW-BTC']
df_btc["timestamp"] = pd.to_datetime(df_btc["event_timestamp"])
df_btc = df_btc.set_index("timestamp")

# 1시간 단위로 리샘플링
df_btc = df_btc.resample("1H").ffill()

# =============================
# 4. Feature Engineering
# =============================
#시계열 특징 생성 시 이동평균 MA_6의 결측치가 없기 위해서는 30시간의 데이터를 확보해야 24시간 치 데이터 계산 가능
df_btc["MA_3"] = df_btc["trade_price"].rolling(3).mean()
df_btc["MA_6"] = df_btc["trade_price"].rolling(6).mean()
df_btc["lag_1"] = df_btc["trade_price"].shift(1)

df_btc = df_btc.dropna()
features = df_btc[["trade_price", "acc_trade_volume", "MA_3", "MA_6", "lag_1"]].values

# =============================
# 5. 입력 시퀀스 만들기 (마지막 24시간)
# =============================
sequence_length = 24
num_features = features.shape[1]

def create_sequences(data, seq_len):
    X = []
    for i in range(len(data) - seq_len+1):
        X.append(data[i:i+seq_len])
    return np.array(X)

X_input = create_sequences(features, sequence_length)

/var/folders/_d/d0lslv953psc0xrjcq10dlk80000gn/T/ipykernel_4175/2061239361.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_btc = df_btc.resample("1H").ffill()


In [3]:
X_input.shape

(1, 24, 5)

# 3. 모델&스케일러 로드

In [4]:
import mlflow
from mlflow.tracking import MlflowClient
import mlflow.keras
mlflow.set_tracking_uri("http://3.39.10.103:5000")
client = MlflowClient()#tracking_uri="http://3.39.10.103:5000")
model_name = "MyLSTMModel_bitcoin_2"
model_version_alias = "champion"
model_info = client.get_model_version_by_alias(model_name, model_version_alias)
model_tags = model_info.tags
run_id = model_info.run_id

# Get the model version using a model URI
model_uri = f"models:/{model_name}@{model_version_alias}"
model = mlflow.keras.load_model(model_uri)




#스케일러 X, y 불러오기
import mlflow
import joblib

scaler_X_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path="scaler_X.joblib"
)
scaler_X = joblib.load(scaler_X_path)

scaler_y_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path="scaler_y.joblib"
)
scaler_y = joblib.load(scaler_y_path)

print("Model and scalers loaded successfully!")

/opt/miniconda3/envs/mlops/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/mlops/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))

Model and scalers loaded successfully!


# 추론

In [5]:
X_input_scaled = scaler_X.transform(X_input.reshape(-1, num_features)).reshape(X_input.shape)

y_pred_scaled = model.predict(X_input_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

print("다음 1시간 예측 가격: {:,.1f} 원".format(y_pred[0][0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
다음 1시간 예측 가격: 158,430,976.0 원
